In [ ]:
from PIL import Image
import copy
import matplotlib.pyplot as plt
import numpy as np
# import pandas as pd
import os
import glob
# from utils import crop_and_mask
# from obb import OBB
import cv2
# from error_analysor import convert_to_world_point, compute_length, compute_segmentation_error
import sys
sys.path.append("/root/simon/volume_estimation/")
# from data_loader import DataGenerator
# import seaborn as sns
%load_ext autoreload
%autoreload 2

## Utils functions

In [ ]:
GREEN_RANGE_MIN_HSV = (100, 20, 20)
GREEN_RANGE_MAX_HSV = (185, 255, 255)

In [ ]:
def rgb_to_hsv(r, g, b):
    maxc = max(r, g, b)
    minc = min(r, g, b)
    v = maxc
    if minc == maxc:
        return 0.0, 0.0, v
    s = (maxc-minc) / maxc
    rc = (maxc-r) / (maxc-minc)
    gc = (maxc-g) / (maxc-minc)
    bc = (maxc-b) / (maxc-minc)
    if r == maxc:
        h = bc-gc
    elif g == maxc:
        h = 2.0+rc-bc
    else:
        h = 4.0+gc-rc
    h = (h/6.0) % 1.0
    return h, s, v

In [ ]:
def crop_and_mask(im, min_hsv=GREEN_RANGE_MIN_HSV, max_hsv=GREEN_RANGE_MAX_HSV):
    """remove green pixels"""
    # im = Image.open(image_path)
    # im = im.convert('RGBA')
    pix = im.load()
    width, height = im.size
    for x in range(width):
        for y in range(height):
            r, g, b = pix[x, y]
            h_ratio, s_ratio, v_ratio = rgb_to_hsv(r / 255.0, g / 255.0, b / 255.0)
            h, s, v = (h_ratio * 360, s_ratio * 255, v_ratio * 255)

            min_h, min_s, min_v = min_hsv
            max_h, max_s, max_v = max_hsv
            if min_h <= h <= max_h and min_s <= s <= max_s and min_v <= v <= max_v:
                pix[x, y] = (0, 0, 0, 0)

    # save the mask
    mask = np.asarray(im)[...,0]
    mask.flags.writeable = True
    mask[mask > 0] = 1
    return mask

## Load images

In [ ]:
left_path = '/root/data/blender/blender_test/Image0028_L.png'
right_path = '/root/data/blender/blender_test/Image0028_R.png'
ground_truth_depth_path = '/root/data/blender/blender_test/true_depth.npy'

In [ ]:
# get ground truth mask + segmentation
mask = crop_and_mask(Image.open(left_path))
ground_truth_depth = np.load(ground_truth_depth_path)
mdepth = ground_truth_depth * mask
x, y = np.nonzero(mdepth>10)
for (i,j) in zip(x,y):
    mask[i,j] = 0
mdepth = ground_truth_depth * mask

In [ ]:
plt.imshow(mask)
plt.show()

# robustness to erosion / dilatation

In [ ]:
from sklearn.cluster import DBSCAN, KMeans

In [ ]:
X = np.stack([np.nonzero(mask)[0], np.nonzero(mask)[1]], axis=1)

In [ ]:
# from sklearn.cluster import DBSCAN
# from sklearn import metrics
# from sklearn.datasets.samples_generator import make_blobs
# from sklearn.preprocessing import StandardScaler

# centers = [[1, 1], [-1, -1], [1, -1]]
# X, labels_true = make_blobs(n_samples=750, centers=centers, cluster_std=0.4,
#                             random_state=0)

In [ ]:
# 
# db = DBSCAN(eps=1.0, min_samples=5).fit(X)

In [ ]:
# # core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
# core_samples_mask[db.core_sample_indices_] = True
# labels = db.labels_

In [ ]:
# import matplotlib.pyplot as plt

# # Black removed and is used for noise instead.
# unique_labels = set(labels)
# colors = [plt.cm.Spectral(each)
#           for each in np.linspace(0, 1, len(unique_labels))]
# for k, col in zip(unique_labels, colors):
#     if k == -1:
#         # Black used for noise.
#         col = [0, 0, 0, 1]

#     class_member_mask = (labels == k)

#     xy = X[class_member_mask & core_samples_mask]
#     plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
#              markeredgecolor='k', markersize=14)

#     xy = X[class_member_mask & ~core_samples_mask]
#     plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
#              markeredgecolor='k', markersize=6)


In [ ]:
random_state = 170
y_pred = KMeans(n_clusters=2, random_state=random_state).fit_predict(X)

In [ ]:
centroids = []
for label in np.unique(y_pred):
    x_mean = np.mean(X[y_pred==label, 0])
    y_mean = np.mean(X[y_pred==label, 1])
    centroids.append((x_mean, y_mean))
true_centroids = np.array(centroids)

In [ ]:
plt.scatter(X[:, 1], X[:, 0], c=y_pred)
plt.scatter(true_centroids[:, 1], true_centroids[:, 0], c='r')
plt.gca().invert_yaxis()
plt.title("Centroids")
plt.show()

In [ ]:
centroid0_x = []
centroid0_y = []
centroid1_x = []
centroid1_y = []
distances = []
for s in range(0, 23):
    if s > 0:
        kernel = np.ones((s, s))
        eroded_mask = cv2.erode(mask, kernel)
    else:
        eroded_mask = mask
    X = np.stack([np.nonzero(eroded_mask)[0], np.nonzero(eroded_mask)[1]], axis=1)
    y_pred = KMeans(n_clusters=2, random_state=random_state).fit_predict(X)
    centroids = []
    for label in np.unique(y_pred):
        x_mean = np.mean(X[y_pred==label, 0])
        y_mean = np.mean(X[y_pred==label, 1])
        centroids.append((x_mean, y_mean))
    centroids = np.array(centroids)
    
    # plot mask + centroids
    plt.scatter(X[:, 1], X[:, 0], c=y_pred)
    plt.scatter(centroids[:, 1], centroids[:, 0], c='r')
    centroid0_x.append(max(centroids[:, 1]))
    centroid0_y.append(max(centroids[:, 0]))
    centroid1_x.append(min(centroids[:, 1]))
    centroid1_y.append(min(centroids[:, 0]))
    
    dist = np.linalg.norm(centroids[0, :] - centroids[1, :])
    distances.append(dist)
    plt.gca().invert_yaxis()
    plt.title("Centroids")
    plt.show()

In [ ]:
plt.plot(distances)
plt.title('inter-centroids distance')

In [ ]:
np.max(distances) / np.min(distances)

In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(X[:, 1], X[:, 0], c=y_pred)
plt.scatter(true_centroids[:, 1], true_centroids[:, 0], c='r', s=200)
plt.scatter(centroid0_x, centroid1_y, c='k', alpha = 0.5, marker='x', linewidths=10, s=100)
plt.scatter(centroid1_x, centroid0_y, c='w', alpha = 0.5, marker='x', linewidths=10, s=100)
plt.gca().invert_yaxis()
plt.title("Centroids")
plt.show()

In [ ]:
plt.plot(centroid0_x)
plt.plot(centroid0_y)
plt.plot(centroid1_x)
plt.plot(centroid1_y)
plt.xlabel('Erosion')